<a href="https://colab.research.google.com/github/mshsu/stat400-sets/blob/main/crashes_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Query the Cal Poly police logs site for HTML source
response = requests.get("https://afd.calpoly.edu/police/campus-reports/logs")
# Create HTML parser
soup = BeautifulSoup(response.content, "html.parser")
# The data is found in an element denoted by <div class="row align-justify"><\div>
# Within that, it is found in a <table><\table> element
div = soup.find("div", {"class": "row align-justify"})
table = div.find("table")

# Initialize list of data row
rows = []
# Iterate through each row of the table to convert from HTML to dict
# Then dd to rows list
for tr in table.find_all("tr"):
  info = tr.find_all("td")
  if len(info) >= 5:
    # The `Details` column has multiple parts of information
    # This code splits up the `Details` column into other appropriate columns
    details = str(info[3]).replace("<td>", "").replace("</strong>", "").replace("<br/>", "").replace("</td", "").split("<strong>")[1:]
    dets = {"Incident": '', "CAD #": '', "Report #": ''}
    for detail in details:
      detail = detail.split(": ")
      dets[detail[0]] = detail[1]
    # Add info to row dictionary, then add to list
    rows.append({
        "Date/Time Reported": info[0].text,
        "Date/Time Occurred": info[1].text,
        "Location": info[2].text,
        "Incident": dets['Incident'],
        "CAD #": dets['CAD #'].replace(">", ""),
        "Report #": dets['Report #'].replace(">", ""),
        "Disposition": info[4].text
    })

# Turn rows list into tabular dataframe
Blotter = pd.DataFrame(rows)
# Export to CSV file
Blotter.to_csv("Blotter.csv", index=False)
# Print here
Blotter

,Date/Time Reported,Date/Time Occurred,Location,Incident,CAD #,Report #,Disposition
0,04/16/2023 12:17 pm,,Pcv Huasna,ALARM - Fire Alarm,23-04-16-005242,,
1,04/16/2023 11:51 am,,Pcv Dover,ALARM - Fire Alarm,23-04-16-005241,,False/Cooking related
2,04/16/2023 11:46 am,04/16/2023 11:46 am - 04/16/2023 11:49 am,Sequoia Hall,VANDALISM - Malicious Mischief report,,230478,
3,04/16/2023 11:46 am,,Sequoia Hall,VANDALISM - Malicious Mischief report,23-04-16-005240,230478,REPORT TAKEN
4,04/16/2023 11:29 am,,Pcv Inyo,FIRE - Fire,23-04-16-005239,,False/Cooking related
...,...,...,...,...,...,...,...
2030,02/14/2023 04:03 pm,02/14/2023 02:05 pm - 02/14/2023 04:00 pm,Education Building,LARCENY-THEFT - PETTY THEFT,,230185,Closed
2031,02/14/2023 04:03 pm,,Education Building,LARCENY-THEFT - Theft Report,23-02-14-002094,230185,REPORT TAKEN
2032,02/14/2023 03:49 pm,,Pcv Huasna,ALARM - Fire Alarm,23-02-14-002091,,False/Cooking related
2033,02/14/2023 03:37 pm,,Pcv Gypsum,ALARM - Fire Alarm,23-02-14-002090,,False/Cooking related
